In [1]:
print("ok")

ok


In [2]:
%pwd

'e:\\AI\\AI,ML Career\\Medical-ChatBot\\research'

In [3]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
import os

def load_pdf_file(data):
    if not os.path.exists(data):
        os.makedirs(data, exist_ok=True)
        print(f"Created missing directory: {data} (put your PDFs here!)")

    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [11]:
extracted_data = load_pdf_file(data='Data/')

In [12]:
extracted_data

[Document(metadata={'producer': 'Acrobat Distiller 5.0.5 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-08-04T16:42:18+00:00', 'author': 'tyoung', 'moddate': '2005-11-11T11:36:13-06:00', 'title': 'MAT_TIP43_Cover_BK-SPN_01AUG05.qxp', 'source': 'Data\\Medical_Book.pdf', 'total_pages': 356, 'page': 0, 'page_label': '1'}, page_content='A Treatment \nImprovement \nProtocol\nMedication-Assisted Treatment\nFor Opioid Addiction in \nOpioid Treatment Programs\nTIP\n43\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nSubstance Abuse and Mental Health Services Administration\nCenter for Substance Abuse Treatment\nwww.samhsa.gov\nMEDICATION-\nASSISTED\nTREATMENT'),
 Document(metadata={'producer': 'Acrobat Distiller 5.0.5 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-08-04T16:42:18+00:00', 'author': 'tyoung', 'moddate': '2005-11-11T11:36:13-06:00', 'title': 'MAT_TIP43_Cover_BK-SPN_01AUG05.qxp', 'source': 'Data\\Medical_Book.pdf', 'total_pages': 35

In [13]:
## chunking operation

In [14]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk

In [16]:
text_chunks = text_split(extracted_data)
print("length of Text chunks:",len(text_chunks))

length of Text chunks: 2358


In [17]:
## download the embeddings from huggingface
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [18]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [21]:
embeddings = download_huggingface_embeddings()

In [22]:
query_result = embeddings.embed_query("Hello world")
print("Length:",len(query_result))

Length: 384


In [23]:
query_result

[-0.010300815105438232,
 0.18307922780513763,
 0.03081119991838932,
 0.0044528404250741005,
 -0.0273361224681139,
 -0.033562567085027695,
 0.03763148933649063,
 -0.031573336571455,
 -0.003391013015061617,
 -0.008950837887823582,
 0.03803616017103195,
 -0.05129104107618332,
 0.0003682934329845011,
 -0.023727091029286385,
 0.0927102193236351,
 -0.027795813977718353,
 -0.035152580589056015,
 -0.003224145621061325,
 -0.0768178328871727,
 -0.05761215463280678,
 0.07257603853940964,
 0.11128544807434082,
 0.01605846732854843,
 0.015908440575003624,
 -0.08232704550027847,
 0.00700728502124548,
 0.02901311218738556,
 0.0011386689729988575,
 0.1167173683643341,
 -0.032327331602573395,
 -0.032271552830934525,
 -0.0012590086553245783,
 0.10591629147529602,
 0.023600876331329346,
 0.009664947167038918,
 0.09834080189466476,
 0.042936310172080994,
 -0.019547604024410248,
 0.019267888739705086,
 -0.06417104601860046,
 0.023923343047499657,
 -0.05288001894950867,
 -0.026469500735402107,
 0.0055487076

In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medicalbot")

In [29]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [30]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY

In [ ]:
## Embed each chunk and upsert the embeddings into your pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [37]:
## LOad existing index 
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings

)

In [38]:
docsearch

In [39]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [40]:
retrieved_doc = retriever.invoke("what is done by MAT progress")

In [41]:
retrieved_doc

[Document(id='b7c9e134-3ef4-43b7-9368-0b5986228fbf', metadata={'author': 'tyoung', 'creationdate': '2005-08-04T16:42:18+00:00', 'creator': 'PScript5.dll Version 5.2', 'moddate': '2005-11-11T11:36:13-06:00', 'page': 122.0, 'page_label': '123', 'producer': 'Acrobat Distiller 5.0.5 (Windows)', 'source': 'Data\\Medical_Book.pdf', 'title': 'MAT_TIP43_Cover_BK-SPN_01AUG05.qxp', 'total_pages': 356.0}, page_content='intensity of services they need vary through-\nout treatment and should be determined by\nindividual circumstances. For many patients,\nMAT is the entry point for diagnosis and treat-\nment of, or referral for, other health care and\npsychosocial needs. In general, most patients\nneed more intensive treatment services at entry,\nmore diversified services during stabilization,\nand fewer, less intensive services after bench-\nmarks of recovery begin to be met (McLellan et'),
 Document(id='ab9ae46a-2f9e-4516-912e-f0d127b91776', metadata={'author': 'tyoung', 'creationdate': '2005-08-0